### Задание №3
### Объектно-Ориентированное Программирование

Задание состоит в том, чтобы реализовать базовые алгоритмы линейной алгебры с помощью базовых средств объектно-ориентированного программирования Python.

Для решения задачи требуется реализовать класс Matrix с базовыми алгоритмами линейной алгебры:

\_\_mul\_\_ - умножение матриц

\_\_add\_\_ - поэлементное сложение матриц

\_\_sub\_\_ - поэлементное вычитание матриц

\_\_pow\_\_ - возведение элементов матрицы в скаларную степень

transpose() - транспонирование матрицы

inverse() - обращение матрицы

det() - определитель матрицы

shape() - размер матрицы (кортеж)

sum() - сумма всех элементов матрицы

а также служебные методы:

\_\_getitem\_\_ - получить элемент по индексу

\_\_setitem\_\_ - задать элемент по индексу

\_\_repr\_\_ и \_\_str\_\_

### Отправка задания

Для сдачи задания необходимо отправить боту @py2021sharebot с указанием seminar03 два файла:
1. result.json
2. seminar03.ipynb

Автоматическая проверка отправки будет реализована командой /check seminar03.

In [1]:
import sys
import os
import math
import copy
import json
import random

Требуется реализовать методы в следующем классе, отмеченные #TODO:

In [65]:
assert all((0 <= 3, 12 < 10))
print(0 <= 3, 11 < 10)

AssertionError: 

In [66]:
class Matrix:
    def __init__(self, nrows, ncols, init='zeros'):
        """
            nrows - количество строк матрицы
            ncols - количество столбцов матрицы
            init - метод инициализации элементов матрицы:
                "zeros" - инициализация нулями
                "ones" - инициализация единицами
                "random" - случайная инициализация
                "eye" - матрица с единицами на главной диагонали
        """
        assert ncols > 0 and ncols > 0
        
        self.nrows = nrows
        self.ncols = ncols

        if init == 'zeros':
            self.data = [[0.] * ncols for _ in range(nrows)]
        elif init == 'ones':
            self.data = [[1.] * ncols for _ in range(nrows)]
        elif init == 'random':
            self.data = [[random.random()] * ncols for _ in range(nrows)]
        elif init == 'eye':
            self.data = [[0.] * ncols for _ in range(nrows)]
            for i in range(min(nrows, ncols)):
                self.data[i][i] = 1.
        else:
            raise ValueError("Inappropriate initialization type.")
    
    @staticmethod
    def fromDict(data):
        "Десериализация матрицы из словаря"
        ncols = data["ncols"]
        nrows = data["nrows"]
        items = data["data"]
        assert len(items) == ncols*nrows
        m = Matrix(nrows, ncols)
        for row in range(nrows):
            for col in range(ncols):
                m[(row, col)] = items[ncols*row + col]
        return m
    
    @staticmethod
    def toDict(M):
        "Сериализация матрицы в словарь"
        assert isinstance(M, Matrix)
        nrows, ncols = M.shape()
        data = []
        for row in range(nrows):
            for col in range(ncols):
                data.append(M[(row, col)])
        return {"nrows": nrows, "ncols": ncols, "data": data}
    
    def __str__(self):
        "Строковое представление матрицы"
        res = ""
        maxLen = 0

        # Get max length of element string
        for i in range(self.nrows):
            for j in range(self.ncols):
                txt = "{0:.4f}".format(float(self[(i, j)]))
                l = len(txt)
                if l > maxLen:
                    maxLen = l

        # Get matrix string
        for i in range(self.nrows):
            if i > 0:
                res += '\n'
            res += '['
            for j in range(self.ncols):
                if j > 0:
                    res += ' '
                txt = "{0:.4f}".format(float(self[(i, j)]))
                l = len(txt)
                for k in range(maxLen - l):
                    res += ' '
                res += txt
            res += ']'
        
        return res
    
    def __repr__(self):
        return f"Matrix {self.nrows}x{self.ncols}"
    
    def shape(self):
        "Вернуть кортеж размера матрицы (nrows, ncols)"
        return self.nrows, self.ncols
    
    def __getitem__(self, index):
        "Вернуть элемент с индесом index (кортеж (row, col))"
        row, col = index
        assert 0 <= row < self.nrows and 0 <= col < self.ncols
        
        return self.data[row][col]
    
    def __setitem__(self, index, value):
        "Присвоить значение value элементу с индесом index (кортеж (row, col))"
        row, col = index
        assert 0 <= row < self.nrows and 0 <= col < self.ncols
        
        self.data[row][col] = value
    
    def __sub__(self, rhs):
        "Вычесть матрицу rhs и вернуть результат"
        assert isinstance(rhs, Matrix)
        assert self.nrows == rhs.nrows and self.ncols == rhs.ncols

        lhs = copy.deepcopy(self)
        for i in range(lhs.nrows):
            for j in range(lhs.ncols):
                lhs[(i, j)] -= rhs[(i, j)]
        
        return lhs
    
    def __add__(self, rhs):
        "Сложить с матрицей rhs и вернуть результат"
        assert isinstance(rhs, Matrix)
        assert self.nrows == rhs.nrows and self.ncols == rhs.ncols

        lhs = copy.deepcopy(self)
        for i in range(lhs.nrows):
            for j in range(lhs.ncols):
                lhs[(i, j)] += rhs[(i, j)]
        
        return lhs
    
    def __mul__(self, rhs):
        "Умножить на матрицу rhs и вернуть результат"
        assert isinstance(rhs, Matrix)
        assert self.ncols == rhs.nrows

        lhs = Matrix(self.nrows, rhs.ncols)
        for i in range(lhs.nrows):
            for j in range(lhs.ncols):
                elem = 0
                for k in range(self.ncols):
                    elem += self[(i, k)] * rhs[(k, j)]
                lhs[(i, j)] = elem
        
        return lhs
    
    def __pow__(self, power):
        "Возвести все элементы в степень pow и вернуть результат"
        lhs = copy.deepcopy(self)
        for i in range(lhs.nrows):
            for j in range(lhs.ncols):
                lhs[(i, j)] **= power

        return lhs
    
    def sum(self):
        "Вернуть сумму всех элементов матрицы"
        sum_ = 0
        for i in range(self.nrows):
            for j in range(self.ncols):
                sum_ += self[(i, j)]

        return sum_
        
    def det(self):
        "Вычислить определитель матрицы"
        assert self.nrows == self.ncols
        
        lhs = copy.deepcopy(self)
        det_ = 1.

        # Perform Gaussian elimination
        for i in range(self.nrows):
            # Eliminate
            scale = 1 / lhs[(i, i)]
            for j in range(i + 1, self.nrows):
                s = scale * lhs[(j, i)]
                for k in range(self. ncols):
                    lhs[(j, k)] -= s * lhs[(i, k)]

            # Update determinant
            det_ *= lhs[(i, i)]

        return det_

    
    def transpose(self):
        "Транспонировать матрицу и вернуть результат"
        lhs = Matrix(self.ncols, self.nrows)
        for i in range(lhs.nrows):
            for j in range(lhs.ncols):
                lhs[(i, j)] = self[(j, i)]

        return lhs
    
    def inv(self):
        "Вычислить обратную матрицу и вернуть результат"
        assert self.nrows == self.ncols
        
        lhs = copy.deepcopy(self)
        rhs = Matrix(self.nrows, self.ncols, init='eye')

        # Perform Gaussian elimination
        for i in range(self.nrows):
            # Scale
            scale = 1 / lhs[(i, i)]
            for j in range(self.ncols):
                lhs[(i, j)] *= scale
                rhs[(i, j)] *= scale

            # Eliminate
            for j in range(self.nrows):
                if j == i:
                    continue

                s = lhs[(j, i)]
                for k in range(self. ncols):
                    lhs[(j, k)] -= s * lhs[(i, k)]
                    rhs[(j, k)] -= s * rhs[(i, k)]
        
        return rhs

In [4]:
def load_file(filename):
    with open(filename, "r") as f:
        input_file = json.load(f)
        A = Matrix.fromDict(input_file["A"])
        B = Matrix.fromDict(input_file["B"])
    return A, B

Проверка реализованных методов

In [68]:
filename = "input/input_028.json"
A, B = load_file(filename)
print("Матрица A: ")
print(A)
print("Матрица B: ")
print(B)
C = A*B
print("Матрица C = A*B: ")
print(C)
C_t = C.transpose()
print("Транспонированная матрица C: ")
print(C_t)
C_inv = C.inv()
print("Матрица, обратная C: ")
print(C_inv)
E = Matrix(C_inv.ncols, C_inv.nrows, init="eye")
D = C_inv + E
print("Матрица D, равная сумме C и единичной матрицы: ")
print(D)
D_det = D.det()
print("Определитель матрицы D: ", D_det)
D_norm = (D**2).sum()**0.5
print("Норма Фробениуса матрицы D: ", D_norm)

Матрица A: 
[0.9945 0.5046 0.2764 0.3067]
[0.4313 0.0533 0.7549 0.9519]
[0.3215 0.0771 0.7294 0.1419]
Матрица B: 
[0.9337 0.3706 0.6405]
[0.1725 0.7399 0.1694]
[0.1241 0.8484 0.2309]
[0.7561 0.1257 0.1171]
Матрица C = A*B: 
[1.2817 1.0150 0.8222]
[1.2253 0.9594 0.5711]
[0.5112 0.8128 0.4040]
Транспонированная матрица C: 
[1.2817 1.2253 0.5112]
[1.0150 0.9594 0.8128]
[0.8222 0.5711 0.4040]
Матрица, обратная C: 
[-0.6879  2.3198 -1.8791]
[-1.8239  0.8754  2.4747]
[ 4.5400 -4.6968 -0.1255]
Матрица D, равная сумме C и единичной матрицы: 
[ 0.3121  2.3198 -1.8791]
[-1.8239  1.8754  2.4747]
[ 4.5400 -4.6968  0.8745]
Определитель матрицы D:  33.80415443202358
Норма Фробениуса матрицы D:  8.087871269303998


Сохранение результатов в файл. Не изменяйте этот код. Отправьте файл result.json в ответ на это задание.

In [69]:
A_dict = Matrix.toDict(A)
B_dict = Matrix.toDict(B)
C_dict = Matrix.toDict(C)
Ct_dict = Matrix.toDict(C_t)
Cinv_dict = Matrix.toDict(C_inv)
result = {
    "A": A_dict,
    "B": B_dict,
    "C": C_dict,
    "Ct": Ct_dict,
    "Cinv": Cinv_dict,
    "D_det": D_det,
    "D_norm": D_norm
}

In [70]:
def save_file(filename, data):
    with open(filename, "w") as f:
        input_file = json.dump(data, f)

In [72]:
save_file("output/result.json", result)